# 04. Sensor Looks and Residuals

How to generate sensor looks and/or residuals.  This includes figuring out when the sun is down at a site and how to just test those times.

In [1]:
import numpy as np
import pandas as pd

import public_astrostandards as PA
import public_astrostandards_tools as PAT

In [2]:
# shared libs no longer need to be init'd, but this will setup a log in "aslog.txt"
PA.init_all( verbose=False )

0

In [3]:
# assume we have a file stored from the first example; this sets up time constants
PAT.astro_time.load_time_constants( './reduced_time_constants.dat', PA )

### Build dates

In [4]:
# set some dates as datetime objects
dates = pd.date_range( '2026-1-1', '2026-1-10', freq='5min')

# now annotate key astrostandards data; again, note that the harness we pass in is for the public_astrostandards (not public_astrostandards_tools)
dates_df = PAT.astro_time.convert_times( dates, PA )
# these should now have key astrostandard fields that will be used in later calls
dates_df.head(5)

,datetime,theta,ds50_utc,ds50_et,ds50_ut1
0,2026-01-01 00:00:00,1.756867,27760.000000,27760.000801,27760.000001
1,2026-01-01 00:05:00,1.778743,27760.003472,27760.004273,27760.003473
2,2026-01-01 00:10:00,1.800620,27760.006944,27760.007745,27760.006945
3,2026-01-01 00:15:00,1.822496,27760.010417,27760.011217,27760.010417
4,2026-01-01 00:20:00,1.844372,27760.013889,27760.014690,27760.013889


### Truth and test TLE

`propTLE_df` will throw an error if astrostandards cannot find the SGP4_Open_License.txt file

In [5]:
ISS = ('1 25544U 98067A   26007.40340206  .00008503  00000-0  16139-3 0  9993','2 25544  51.6331  15.4118 0007618 351.0107   9.0743 15.49153097546833')
# we'll modify inclination and mean motion to make this a fake TLE
fake_ISS = ('1 99999U 98067A   26007.40340206  .00008503  00000-0  16139-3 0  9993','2 99999  50.0000  15.4118 0007618 351.0107   9.0743 15.00000097546833')

print('So you can compare')
for A,B in zip( ISS, fake_ISS ):
    print(A)
    print(B)
    print()

# this will throw an error if astrostandards cannot find the SGP4_Open_License.txt file
actual_eph = PAT.sgp4.propTLE_df( dates_df.copy(), *ISS, PA )
test_eph   = PAT.sgp4.propTLE_df( dates_df.copy(), *fake_ISS, PA )

So you can compare
1 25544U 98067A   26007.40340206  .00008503  00000-0  16139-3 0  9993
1 99999U 98067A   26007.40340206  .00008503  00000-0  16139-3 0  9993

2 25544  51.6331  15.4118 0007618 351.0107   9.0743 15.49153097546833
2 99999  50.0000  15.4118 0007618 351.0107   9.0743 15.00000097546833



## Ground site and sundown

In [6]:
# sensor location : lat, lon, altitude (km)
sen_lla = (38.83, -104.82, 1.832 )
# build a sensor ground site frame (note that it has some common columns, like "teme_p"
sensor_f = PAT.sensor.setup_ground_site( dates_df.copy(), *sen_lla, PA )
sensor_f.head(5)

,datetime,theta,ds50_utc,ds50_et,ds50_ut1,lat,lon,height,teme_p
0,2026-01-01 00:00:00,1.756867,27760.000000,27760.000801,27760.000001,38.83,-104.82,1.832,"[4963.502978536777, -360.9208206121891, 3978.7..."
1,2026-01-01 00:05:00,1.778743,27760.003472,27760.004273,27760.003473,38.83,-104.82,1.832,"[4970.210322229878, -252.25980554659574, 3978...."
2,2026-01-01 00:10:00,1.800620,27760.006944,27760.007745,27760.006945,38.83,-104.82,1.832,"[4974.539144491277, -143.4780701669443, 3978.7..."
3,2026-01-01 00:15:00,1.822496,27760.010417,27760.011217,27760.010417,38.83,-104.82,1.832,"[4976.487373738568, -34.62767268296774, 3978.7..."
4,2026-01-01 00:20:00,1.844372,27760.013889,27760.014690,27760.013889,38.83,-104.82,1.832,"[4976.054077640019, 74.23929617780217, 3978.78..."


In [7]:
# now compute where the sun is
sun_f = dates_df.copy()
sun_f['teme_p'] = PAT.sensor.sun_at_time( sun_f, PA ) 

# compute looks from the sensor to the sun
looks_f = PAT.sensor.compute_looks( sensor_f, sun_f, PA )

# find those times when the sun is down; NOTE we're indexing subsets of the DATE and SENSOR frame
idx = looks_f['XA_TOPO_EL'] < -4
sensor_sundown_f = sensor_f[ idx  ].copy()

# get a copy of the slice of times
sensor_sundown_dates = sensor_sundown_f[ PAT.astro_time.DATE_FIELDS ].copy()
sensor_sundown_dates.head(5)

,datetime,theta,ds50_utc,ds50_et,ds50_ut1
2,2026-01-01 00:10:00,1.800620,27760.006944,27760.007745,27760.006945
3,2026-01-01 00:15:00,1.822496,27760.010417,27760.011217,27760.010417
4,2026-01-01 00:20:00,1.844372,27760.013889,27760.014690,27760.013889
5,2026-01-01 00:25:00,1.866249,27760.017361,27760.018162,27760.017362
6,2026-01-01 00:30:00,1.888125,27760.020833,27760.021634,27760.020834


In [8]:
# now we know when the sun is down at site (note: this is just sampled, you could interpolate the XA_TOPO_EL  field to find *exact* sundown times and drill down
# note the use of idx: we will align the sensor and target frames, and get looks during those times
# do this for both test and actual
sensor_to_actual_df = PAT.sensor.compute_looks( sensor_sundown_f, actual_eph[idx], PA )
sensor_to_test_df   = PAT.sensor.compute_looks( sensor_sundown_f, test_eph[idx], PA )

sensor_to_test_df.head(5)

,datetime_sensor,theta_sensor,ds50_utc_sensor,ds50_et_sensor,ds50_ut1_sensor,lat_sensor,lon_sensor,height_sensor,teme_p_sensor,astrolat_sensor,...,XA_TOPO_RA,XA_TOPO_DEC,XA_TOPO_AZ,XA_TOPO_EL,XA_TOPO_RANGE,XA_TOPO_RADOT,XA_TOPO_DECDOT,XA_TOPO_AZDOT,XA_TOPO_ELDOT,XA_TOPO_RANGEDOT
0,2026-01-01 00:10:00,1.800620,27760.006944,27760.007745,27760.006945,38.83,-104.82,1.832,"[4974.539144491277, -143.4780701669443, 3978.7...",39.017367,...,88.122237,-43.261771,126.266479,-25.418495,6578.480420,0.036822,0.057302,-0.067996,0.008770,-1.445740
1,2026-01-01 00:15:00,1.822496,27760.010417,27760.011217,27760.010417,38.83,-104.82,1.832,"[4976.487373738568, -34.62767268296774, 3978.7...",39.017367,...,100.033030,-26.078013,105.149206,-23.772487,6316.204070,0.042766,0.056042,-0.071973,0.001901,-0.252714
2,2026-01-01 00:20:00,1.844372,27760.013889,27760.014690,27760.013889,38.83,-104.82,1.832,"[4976.054077640019, 74.23929617780217, 3978.78...",39.017367,...,113.804243,-10.494360,83.706610,-24.365061,6440.201727,0.048899,0.046408,-0.069893,-0.005812,1.070005
3,2026-01-01 00:25:00,1.866249,27760.017361,27760.018162,27760.017362,38.83,-104.82,1.832,"[4973.239463552772, 183.0707373016704, 3978.78...",39.017367,...,129.156935,1.084385,63.725768,-27.136573,6935.340988,0.052876,0.030093,-0.062805,-0.012343,2.170712
4,2026-01-01 00:30:00,1.888125,27760.020833,27760.021634,27760.020834,38.83,-104.82,1.832,"[4968.044878419939, 291.8145690335834, 3978.78...",39.017367,...,145.096034,7.443292,46.108249,-31.555842,7700.454817,0.052601,0.012538,-0.054812,-0.016765,2.856707


In [9]:
# we can calculate the residuals for RA, DEC, AZ, EL, etc...
residuals = pd.DataFrame()
for col in ['XA_TOPO_AZ','XA_TOPO_EL','XA_TOPO_RA','XA_TOPO_DEC']:
    residuals[col] = sensor_to_actual_df[col] - sensor_to_test_df[col]
residuals.head(5)

,XA_TOPO_AZ,XA_TOPO_EL,XA_TOPO_RA,XA_TOPO_DEC
0,48.115159,-12.135570,-16.375109,-42.092741
1,52.507123,-8.592426,-23.734430,-44.655705
2,55.399900,-3.463388,-28.685926,-43.924349
3,54.406218,2.458692,-33.146280,-37.649054
4,49.324325,7.848294,-35.940923,-26.134925


Horizon check

In [10]:
# look at the output for the actual.. is it ever above the horizon?
above_horizon_idx =  sensor_to_actual_df['XA_TOPO_EL'] > 5 
residuals[ above_horizon_idx ]

,XA_TOPO_AZ,XA_TOPO_EL,XA_TOPO_RA,XA_TOPO_DEC
61,56.956954,17.575058,-48.568353,-27.009631
80,6.399707,-6.785129,-9.584150,0.965011
81,-3.097997,-7.973818,11.573452,-3.582372
100,-305.505849,0.393914,285.192457,27.640152
119,37.007478,15.981587,-21.743943,35.979148
...,...,...,...,...
1394,-44.167485,24.626890,20.378171,46.917845
1413,281.699141,13.487947,81.169869,54.032804
1432,263.132655,-31.338258,-191.199886,1.247406
1433,-27.548421,17.351852,3.378976,31.851627
